In [0]:
# List all tables in the default workspace
spark.sql("SHOW TABLES IN default").show()


+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|social_media_sent...|      false|
| default|social_media_sent...|      false|
| default|social_media_sent...|      false|
| default|social_media_sent...|      false|
+--------+--------------------+-----------+



In [0]:
# Load the table
sent_df = spark.table("workspace.default.social_media_sentiment_target_80")

# OR equivalently
# sent_df = spark.sql("SELECT * FROM workspace.default.social_media_sentiment_target_80")

# Preview schema and sample rows
sent_df.printSchema()
display(sent_df.limit(10))


root
 |-- tweet_id: long (nullable = true)
 |-- username: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- sentiment: string (nullable = true)



tweet_id username tweet_text sentiment 1 Vivaan Iyer Good functional banking :/ Neutral 2 Vivaan Reddy Good experience but sometimes hardly loved :) Neutral 3 Arjun Singh login feels loved though crashes happens! Positive 4 Ishita Sharma Poor support but sometimes hardly buggy :) Positive 5 Sneha Joshi app feels crashes though excellent happens!! Negative 6 Vihaan Chopra easy banking is Poor! Neutral 7 Ishita Iyer Mostly reliable, app is okay!! Positive 8 Krishna Iyer The app is terrible and Mixed :/ Negative 9 Diya Reddy Decent easy support :) Positive 10 Aditya Agarwal app feels regular though hardly buggy happens Positive

In [0]:
from pyspark.sql.functions import trim, col

# Remove leading/trailing spaces and null tweets
sent_df = sent_df.withColumn("tweet_text", trim(col("tweet_text"))) \
                 .withColumn("username", trim(col("username"))) \
                 .filter(col("tweet_text").isNotNull())

print("Total rows:", sent_df.count())
display(sent_df.groupBy("sentiment").count())


Total rows: 500


sentiment count Neutral 141 Positive 193 Negative 166

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml import Pipeline

# Convert sentiments (Positive/Negative/Neutral) to numeric labels
indexer = StringIndexer(inputCol="sentiment", outputCol="label").fit(sent_df)
tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words_raw")
stop_remover = StopWordsRemover(inputCol="words_raw", outputCol="words")
cv = CountVectorizer(inputCol="words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")

# Build pipeline
pipeline = Pipeline(stages=[indexer, tokenizer, stop_remover, cv, idf])
prep_model = pipeline.fit(sent_df)
prep_df = prep_model.transform(sent_df)

display(prep_df.select("tweet_text", "words", "features", "label").limit(5))


tweet_text words features label Good functional banking :/ List(good, functional, banking, :/) {"type":"0","size":"167","indices":["4","13","20","32"],"values":["1.8098868538206117","2.409943611314545","2.503034034380557","2.920769235080536"]} 2.0 Good experience but sometimes hardly loved :) List(good, experience, sometimes, hardly, loved, :)) {"type":"0","size":"167","indices":["3","6","10","20","25","36"],"values":["1.7977654932882667","1.9399399820688095","2.2847804683605393","2.503034034380557","2.7508701982851385","2.958509563063383"]} 2.0 login feels loved though crashes happens! List(login, feels, loved, though, , crashes, happens!) {"type":"0","size":"167","indices":["0","1","2","11","16","25","47"],"values":["1.6114359150967734","1.6114359150967734","1.7392692866066584","2.3045830956567186","2.4324164671666035","2.7508701982851385","3.4440173788450834"]} 0.0 Poor support but sometimes hardly buggy :) List(poor, support, sometimes, hardly, buggy, :)) {"type":"0","size":"167","indices":["3","6","23","24","34","36"],"values":["1.7977654932882667","1.9399399820688095","2.579019941358479","2.6902455764687034","2.958509563063383","2.958509563063383"]} 0.0 app feels crashes though excellent happens!! List(app, feels, crashes, though, , excellent, happens!!) {"type":"0","size":"167","indices":["0","1","2","9","16","41","45"],"values":["1.6114359150967734","1.6114359150967734","1.7392692866066584","2.2276220545205905","2.4324164671666035","3.0385522707369192","3.2721671219184243"]} 1.0

In [0]:
train_df, test_df = prep_df.randomSplit([0.8, 0.2], seed=42)
print("Train:", train_df.count(), "Test:", test_df.count())


Train: 426 Test: 74


In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=30)
lr_model = lr.fit(train_df)

predictions = lr_model.transform(test_df)
display(predictions.select("tweet_text", "sentiment", "prediction", "probability").limit(10))


tweet_text sentiment prediction probability login feels loved though crashes happens! Positive 0.0 {"type":"1","size":null,"indices":null,"values":["0.8154706355394552","0.02813496708957511","0.1563943973709698"]} Mostly reliable, app is okay!! Positive 0.0 {"type":"1","size":null,"indices":null,"values":["0.9999772829883942","2.2715453566205944E-5","1.5580395735325622E-9"]} Decent easy support :) Positive 1.0 {"type":"1","size":null,"indices":null,"values":["0.2553876519518064","0.7201694414175496","0.024442906630643985"]} Great service but sometimes failed… Negative 2.0 {"type":"1","size":null,"indices":null,"values":["4.793417352214514E-6","1.1294336468407109E-5","0.9999839122461794"]} The UPI is delayed and Terrible… Negative 1.0 {"type":"1","size":null,"indices":null,"values":["1.004463823237729E-6","0.9999962648084715","2.730727705202578E-6"]} Decent excellent login Positive 0.0 {"type":"1","size":null,"indices":null,"values":["0.9623264811269064","0.01869003308823371","0.018983485784859845"]} app feels delayed though easy happens :) Negative 1.0 {"type":"1","size":null,"indices":null,"values":["0.018638354737706964","0.9811852535509572","1.763917113359882E-4"]} The app is smooth and Great! Positive 0.0 {"type":"1","size":null,"indices":null,"values":["0.9999992792307447","2.8332172975098343E-7","4.374475254306461E-7"]} The experience is terrible and Mixed. Negative 1.0 {"type":"1","size":null,"indices":null,"values":["0.07835447987428756","0.9096275362527714","0.012017983872941142"]} Good loved transfer!! Positive 0.0 {"type":"1","size":null,"indices":null,"values":["0.9990970303254605","1.2712358353258502E-9","9.029684033035704E-4"]}

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.5946


In [0]:
display(sent_df.groupBy("sentiment").count())


sentiment count Neutral 141 Positive 193 Negative 166

Databricks visualization. Run in Databricks to view.